The purpose of this notebook is to demonstrate various parameter tunings' impact on baseline model performance. Generally, only a single parameter will be tuned during each test; however, certain parameters will require multiple tunings, such as a 6x6 CNN layer requiring a certain input shape.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
import metric_functions as mf

Using TensorFlow backend.


In [1]:
pip install Pillow --user

     |████████████████████████████████| 2.1MB 1.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras --user

     |████████████████████████████████| 378kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 276kB 1.4MB/s eta 0:00:01
Failed to build pyyaml
    Running setup.py install for pyyaml ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn --user

     |████████████████████████████████| 7.3MB 1.6MB/s eta 0:00:01
     |████████████████████████████████| 307kB 696kB/s eta 0:00:01
Failed to build sklearn
    Running setup.py install for sklearn ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Compile and Create Data <a id='data'></a>

## Identify Working Directory

In [2]:
working_directory = 'mixed_frames'

In [3]:
train_folder = os.path.join('mcu_data', working_directory, 'train')
train_mcu = os.path.join(train_folder, 'mcu')
train_non = os.path.join(train_folder, 'non_mcu')

test_folder = os.path.join('mcu_data', working_directory, 'test')
test_mcu = os.path.join(test_folder, 'mcu')
test_non = os.path.join(test_folder, 'non_mcu')

print('There are', len(os.listdir(train_mcu)) + len(os.listdir(train_non)), 'images in the Train set')
print('There are', len(os.listdir(test_mcu)) + len(os.listdir(test_non)), 'images in the Test set')

print('\n')

print('There are', len(os.listdir(train_mcu)), 'MCU images in the train set')
print('There are', len(os.listdir(train_non)), 'non-MCU images in the train set')
print('There are', len(os.listdir(test_mcu)), 'MCU images in the test set')
print('There are', len(os.listdir(test_non)), 'non-MCU images in the test set')

There are 5214 images in the Train set
There are 1852 images in the Test set


There are 1916 MCU images in the train set
There are 3298 non-MCU images in the train set
There are 478 MCU images in the test set
There are 1374 non-MCU images in the test set


## Create input data with image size of 185 x 100

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size = (185, 100),
    batch_size = 4148,
    color_mode = 'grayscale',
    subset = 'training')

val_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size = (185, 100),
    batch_size = 4148,
    color_mode = 'grayscale',
    subset = 'validation')


Found 3031 images belonging to 2 classes.
Found 756 images belonging to 2 classes.


In [6]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_folder, target_size=(185, 100), color_mode = 'grayscale', batch_size = 1781)

Found 1718 images belonging to 2 classes.


In [7]:
class_labels = ['Non-MCU', 'MCU']
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)
train_y = np.reshape(train_labels[:,0], (3031,1))
val_y = np.reshape(val_labels[:,0], (756,1))
test_y = np.reshape(test_labels[:,0], (1718,1))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

train_images shape: (3031, 185, 100, 1)
train_labels shape: (3031, 2)
val_images shape: (756, 185, 100, 1)
val_labels shape: (756, 2)
test_images shape: (1718, 185, 100, 1)
test_labels shape: (1718, 2)


### Create input data with image size of 128 x 128

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size = (128, 128),
    batch_size = 4172,
    color_mode = 'grayscale',
    subset = 'training')

val_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size = (128, 128),
    batch_size = 1042,
    color_mode = 'grayscale',
    subset = 'validation')


Found 4172 images belonging to 2 classes.
Found 1042 images belonging to 2 classes.


In [13]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_folder, target_size=(128, 128), color_mode = 'grayscale', batch_size = 1852)

Found 1852 images belonging to 2 classes.


In [14]:
class_labels = ['Non-MCU', 'MCU']
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)
train_y = np.reshape(train_labels[:,0], (4172,1))
val_y = np.reshape(val_labels[:,0], (1042,1))
test_y = np.reshape(test_labels[:,0], (1852,1))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

train_images shape: (4172, 128, 128, 1)
train_labels shape: (4172, 2)
val_images shape: (1042, 128, 128, 1)
val_labels shape: (1042, 2)
test_images shape: (1852, 128, 128, 1)
test_labels shape: (1852, 2)


# Tuning <a id='Tuning'></a>
